# Catchement area of farms

## define api key

In [11]:
api_key = "5b3ce3597851110001cf62481aa22375f5e64d2d8d68d87265d46482"

## load data

In [12]:
import geopandas as gpd

gdf = gpd.read_file('../data/processed_data/top_500_buildings_with_farms_details.geojson')
print(gdf.shape)
gdf.head()

(500, 9)


,egid,buildingStatus,buildingCategory,buildingClass,municipalityNumber,municipalityName,nearby_farms,total_products,geometry
0,210197731,1004,1060,1252,159,Uetikon am See,"[ { ""farm_id"": 4.0, ""distance"": 5996.964112004...",802,POINT (8.689 47.274)
1,210197448,1004,1060,1252,159,Uetikon am See,"[ { ""farm_id"": 4.0, ""distance"": 5821.287914910...",791,POINT (8.682 47.277)
2,210197590,1004,1060,1252,159,Uetikon am See,"[ { ""farm_id"": 4.0, ""distance"": 5829.916037383...",791,POINT (8.686 47.275)
3,210202495,1004,1060,1252,155,MÃ¤nnedorf,"[ { ""farm_id"": 4.0, ""distance"": 6068.202328155...",788,POINT (8.697 47.268)
4,210213365,1004,1060,1252,157,Oetwil am See,"[ { ""farm_id"": 4.0, ""distance"": 8368.715941914...",782,POINT (8.725 47.276)


## available profiles
* driving-car
* driving-hgv
* cycling-regular
* cycling-road
* cycling-mountain
* cycling-electric
* foot-walking
* foot-hiking
* wheelchair

In [13]:
from openrouteservice import client
import json

# Settings
transport = "cycling-regular"

# Client settings
ors = client.Client(key=api_key)


def create_isochrone(longitude: float, latitude: float, traveltime=5) -> str:
    params_iso = {
        "profile": "cycling-regular",
        "range": [traveltime * 60],
        "attributes": ["total_pop"],
        "locations": [[longitude, latitude]],
    }

    try:
        result = ors.isochrones(**params_iso)
        return json.dumps(result)
    except Exception as e:
        print("Error creating isochrone:", e)
        return json.dumps({"error": str(e)})

In [18]:
# example for Steinmaur 🥳
create_isochrone(8.45216, 47.4971, traveltime=15)

'{"type": "FeatureCollection", "bbox": [8.398355, 47.46126, 8.505215, 47.533207], "features": [{"type": "Feature", "properties": {"group_index": 0, "value": 900.0, "center": [8.452252935602223, 47.49704224115783], "total_pop": 27405.0}, "geometry": {"coordinates": [[[8.398355, 47.502548], [8.398405, 47.500745], [8.400126, 47.494672], [8.405152, 47.487262], [8.406008, 47.485894], [8.406688, 47.485922], [8.408737, 47.485881], [8.410116, 47.485733], [8.418053, 47.482698], [8.421355, 47.478088], [8.423814, 47.473366], [8.423996, 47.473012], [8.42405, 47.472918], [8.42545, 47.471687], [8.430057, 47.469718], [8.433531, 47.466396], [8.440748, 47.465409], [8.446795, 47.46144], [8.451527, 47.46126], [8.452338, 47.461319], [8.453934, 47.462205], [8.461148, 47.462706], [8.46321, 47.461607], [8.469878, 47.464287], [8.470188, 47.464427], [8.470324, 47.4645], [8.473711, 47.47053], [8.481785, 47.473694], [8.485845, 47.475088], [8.487509, 47.475742], [8.488008, 47.475927], [8.491922, 47.477486], [8.49

## Create isochrones

In [15]:
# create isochrones for all buildings (limit to first 5 for development)
limit = 5
for idx, row in gdf.head(limit).iterrows():
    print(f'Processing building with index "{idx}"')
    try:
        result = create_isochrone(row["geometry"].x, row["geometry"].y)
        gdf.loc[idx, "isochrone"] = result
    except Exception as e:
        print(f"Failed to process building {idx}: {e}")

Processing building with index "0"
Processing building with index "1"
Processing building with index "2"
Processing building with index "3"
Processing building with index "4"


In [16]:
print(gdf.head(n=1))

        egid  buildingStatus  buildingCategory  buildingClass  \
0  210197731            1004              1060           1252   

   municipalityNumber municipalityName  \
0                 159   Uetikon am See   

                                        nearby_farms  total_products  \
0  [ { "farm_id": 4.0, "distance": 5996.964112004...             802   

               geometry                                          isochrone  
0  POINT (8.689 47.274)  {"type": "FeatureCollection", "bbox": [8.67216...  


## display with folium

In [17]:
import folium
import json

# Step 1: Create a base map
# Let's say the first building's coordinates are used to center the map
if not gdf.empty and "geometry" in gdf.columns:
    first_item = gdf.iloc[0]
    map_center = [first_item["geometry"].y, first_item["geometry"].x]
else:
    map_center = [47.3769, 8.5417]  # Default to Zurich, Switzerland

m = folium.Map(location=map_center, zoom_start=12, tiles="CartoDB positron")

# Step 2: Add isochrones to the map
for idx, row in gdf.head(limit).iterrows():
    if "isochrone" in row and row["isochrone"]:
        try:
            # Parse the JSON string back to a dictionary
            isochrone_data = json.loads(row["isochrone"])

            # Use Folium's GeoJson to add it to the map
            folium.GeoJson(
                isochrone_data,
                name=f"Isochrone {idx}",
                style_function=lambda feature: {
                    "fillColor": "#blue",
                    "color": "blue",
                    "weight": 2,
                    "dashArray": "5, 5",
                    "fillOpacity": 0.5,
                },
            ).add_to(m)
        except json.JSONDecodeError:
            print(f"Failed to decode JSON for row {idx}")

# Add layer control to toggle layers
folium.LayerControl().add_to(m)

# Display the map
m